<a href="https://colab.research.google.com/github/edurso/tfod-wkspc/blob/master/notebooks/test_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf /tf/models /tf/workspace
!mkdir -p /tf/data
from google.colab import drive
import os
drive.mount('/tf/data')

Drive already mounted at /tf/data; to attempt to forcibly remount, call drive.mount("/tf/data", force_remount=True).


In [ ]:
# install deps 
!python --version
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim pycocotools

Python 3.7.12
  Using cached tensorflow-2.8.0rc0-cp37-cp37m-manylinux2010_x86_64.whl (492.1 MB)
     |████████████████████████████████| 352 kB 5.3 MB/s 


In [ ]:
# clear previous runs
!mkdir -p /tf/models/ /tf/workspace/

# clone repos
!git clone --depth 1 https://github.com/tensorflow/models /tf/models
!git clone --depth 1 https://github.com/edurso/tfod-wkspc /tf/workspace

Cloning into '/tf/models'...
remote: Enumerating objects: 3219, done.
remote: Counting objects: 100% (3219/3219), done.
remote: Compressing objects: 100% (2730/2730), done.
remote: Total 3219 (delta 863), reused 1355 (delta 445), pack-reused 0
Receiving objects: 100% (3219/3219), 33.42 MiB | 25.52 MiB/s, done.
Resolving deltas: 100% (863/863), done.
Cloning into '/tf/workspace'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 40 (delta 7), reused 18 (delta 2), pack-reused 0
Unpacking objects: 100% (40/40), done.


In [ ]:
# compile protobufs
%cd /tf/models/research/
!protoc object_detection/protos/*.proto --python_out=.

# install object detector package
!cp /tf/models/research/object_detection/packages/tf2/setup.py . 
!pip install .

/tf/models/research
Processing /tf/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.8 MB 8.4 MB/s 
     |████████████████████████████████| 1.8 MB 35.3 MB/s 
     |████████████████████████████████| 23.1 MB 1.6 MB/s 
     |████████████████████████████████| 47.6 MB 138 kB/s 
     |████████████████████████████████| 213 kB 42.7 MB/s 
     |████████████████████████████████| 90 kB 6.8 MB/s 
     |████████████████████████████████| 99 kB 9.7 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 596 kB 44.9 MB/s 
     |████████████████████████████████

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import zipfile
import tensorflow as tf
print(tf.__version__)

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [ ]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))

model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)
print(detection_model.signatures['serving_default'].inputs)



def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict



def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)